In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 783030539464897729, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6785064830578995904
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 17101927278958023138
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15882446439
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15386819369618420280
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2, l1

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.14.0


In [5]:
batch_size = 64
img_height, img_width = 600, 450
input_shape = (img_height, img_width, 3)
epochs = 1000

In [6]:
print(os.listdir("../input/digestpath-classification/digestpath_classification/digestpath_classification/"))

['test', 'train']


In [7]:
train_dir = '../input/digestpath-classification/digestpath_classification/digestpath_classification/train/'
test_dir = '../input/digestpath-classification/digestpath_classification/digestpath_classification/test/'

In [8]:
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by imagenet mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [9]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
#     featurewise_center=True,
#     featurewise_std_normalization=True,
    preprocessing_function = preprocess_input,
    validation_split= 0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 460 images belonging to 2 classes.
Found 115 images belonging to 2 classes.
Found 85 images belonging to 2 classes.


In [10]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 460
nb_validation_samples: 115
nb_test_samples: 85

predict_size_train: 8
predict_size_validation: 2
predict_size_test: 2

 num_classes: 2


In [11]:
os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "VGG19_descriptors"

In [12]:
vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"
xception_weights = "../input/full-keras-pretrained-no-top/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [13]:
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
from keras.applications import DenseNet121
from keras.applications import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
from keras.applications import MobileNet

In [14]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionResNetV2(weights=inception_resnet_v2_weights, include_top=False, pooling = "avg")

In [15]:
# base_model1.summary()

In [16]:
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

In [17]:
bottleneck_final_model = base_model1

In [18]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [19]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [20]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [ ]:

# import tensorflow as tf
# from keras.backend.tensorflow_backend import get_session
# from keras.backend.tensorflow_backend import clear_session
# from keras.backend.tensorflow_backend import set_session

# def reset_keras_tf_session():
#     """
#     this function clears the gpu memory and set the 
#     tf session to not use the whole gpu
#     """
#     sess = get_session()
#     clear_session()
#     sess.close()
#     sess = get_session()

# #     config = tf.ConfigProto()
# #     config.gpu_options.allow_growth = True
# #     set_session(tf.Session(config=config))

# reset_keras_tf_session()

In [21]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [22]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        'hidden1':hp.choice('hidden1',[4096,2048,1024,512,256]),
#         'hidden2':hp.choice('hidden2',[2048,1024,512,256,128,]),
#         'hidden3':hp.choice('hidden3',[512,256,128,64,32]),    
        'bias_reg': hp.choice('bias_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'act_reg': hp.choice('act_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'ker_reg': hp.choice('ker_reg',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
        'activation_function':hp.choice('activation_function',["relu","elu","selu","softplus","tanh",])
        }

In [23]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
#     dropout_rate = 0.5
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(params["hidden1"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
#     model.add(Dropout(0.25))

#     model.add(Dense(params["hidden2"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))
   
#     model.add(Dense(params["hidden3"], activation=params["activation_function"], kernel_regularizer=l2(params["ker_reg"]), bias_regularizer=l2(params["bias_reg"]), activity_regularizer=l1(params["act_reg"])))

    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation="softmax"))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [24]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [25]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:                                  
{'act_reg': 0.01, 'activation_function': 'selu', 'beta_1': 0.1, 'beta_2': 0.995, 'bias_reg': 0.001, 'hidden1': 1024, 'ker_reg': 1e-06, 'lr': 1e-07}
Train on 460 samples, validate on 115 samples        
Epoch 1/1000                                         
 - 5s - loss: 201.6508 - acc: 0.3761 - val_loss: 176.6645 - val_acc: 0.3826

Epoch 2/1000                                         
 - 0s - loss: 201.3203 - acc: 0.3717 - val_loss: 176.5529 - val_acc: 0.3826

Epoch 3/1000                                         
 - 0s - loss: 201.6175 - acc: 0.3739 - val_loss: 176.4402 - val_acc: 0.3826

Epoch 4/1000                                         
 - 0s - loss: 201.4954 - acc: 0.3674 - val_loss: 176.3272 - val_acc: 0.3826

Epoch 00004: early stopping                          
 64/115 [===============>..............]             
 - ETA: 0s                                           
                                                    
115/1

Epoch 10/1000                                                                 
 - 0s - loss: 0.7631 - acc: 0.5043 - val_loss: 0.6825 - val_acc: 0.6000       

Epoch 11/1000                                                                 
 - 0s - loss: 0.7394 - acc: 0.5196 - val_loss: 0.6804 - val_acc: 0.6000       

Epoch 12/1000                                                                 
 - 0s - loss: 0.7550 - acc: 0.5000 - val_loss: 0.6782 - val_acc: 0.6087       

Epoch 13/1000                                                                 
 - 0s - loss: 0.7322 - acc: 0.5152 - val_loss: 0.6762 - val_acc: 0.6174       

Epoch 14/1000                                                                 
 - 0s - loss: 0.7397 - acc: 0.5283 - val_loss: 0.6742 - val_acc: 0.6174       

Epoch 15/1000                                                                 
 - 0s - loss: 0.7262 - acc: 0.5261 - val_loss: 0.6725 - val_acc: 0.6174       

Epoch 16/1000                                 

 - 0s 167us/step                                                               

Validation Accuracy: 73.9130%                                                  
Validation Loss: 0.6162675930106122                                            
Test Accuracy: 0.7647058823529411                                              
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:                                                            
{'act_reg': 1e-05, 'activation_function': 'selu', 'beta_1': 0.6, 'beta_2': 0.9, 'bias_reg': 0.1, 'hidden1': 256, 'ker_reg': 1e-05, 'lr': 1e-07}
Train on 460 samples, validate on 115 samples                                  
Epoch 1/1000                                                                   
 - 5s - loss: 0.8823 - acc: 0.5022 - val_loss: 0.7184 - val_acc: 0.6087        

Epoch 2/1000                                                   

*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:                                                            
{'act_reg': 1e-07, 'activation_function': 'elu', 'beta_1': 0.5, 'beta_2': 0.7, 'bias_reg': 1e-07, 'hidden1': 512, 'ker_reg': 1e-05, 'lr': 0.0001}
Train on 460 samples, validate on 115 samples                                  
Epoch 1/1000                                                                   
 - 6s - loss: 0.6174 - acc: 0.6739 - val_loss: 0.4961 - val_acc: 0.8348        

Epoch 2/1000                                                                   
 - 0s - loss: 0.4696 - acc: 0.7957 - val_loss: 0.4383 - val_acc: 0.8174        

Epoch 3/1000                                                                   
 - 0s - loss: 0.4137 - acc: 0.8000 - val_loss: 0.4957 - val_acc: 0.7739        

Epoch 4/1000                                                

Train on 460 samples, validate on 115 samples                                  
Epoch 1/1000                                                                   
 - 6s - loss: 0.6556 - acc: 0.6239 - val_loss: 0.5205 - val_acc: 0.7565        

Epoch 2/1000                                                                   
 - 0s - loss: 0.5010 - acc: 0.7848 - val_loss: 0.4489 - val_acc: 0.8261        

Epoch 3/1000                                                                   
 - 0s - loss: 0.4220 - acc: 0.8261 - val_loss: 0.4350 - val_acc: 0.8000        

Epoch 4/1000                                                                   
 - 0s - loss: 0.3866 - acc: 0.8348 - val_loss: 0.4108 - val_acc: 0.8522        

Epoch 5/1000                                                                   
 - 0s - loss: 0.3795 - acc: 0.8326 - val_loss: 0.4549 - val_acc: 0.8087        

Epoch 6/1000                                                                   
 - 0s - loss: 0.3722 - acc: 0.8435 

...

 64/115 [===============>..............]                                           
 - ETA: 0s                                                                         
                                                                                  
115/115 [==============================]                                           
 - 0s 219us/step                                                                   

Validation Accuracy: 70.4348%                                                      
Validation Loss: 0.5420386910438537                                                
Test Accuracy: 0.7529411764705882                                                  
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:                                                                
{'act_reg': 0.001, 'activation_function': 'selu', 'beta_1': 0.5, 'beta_2': 0.9, 'bias_reg': 1

Validation Loss: 31.94250786822775                                                 
Test Accuracy: 0.7294117647058823                                                  
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:                                                                
{'act_reg': 1e-06, 'activation_function': 'selu', 'beta_1': 0.7, 'beta_2': 0.995, 'bias_reg': 0.1, 'hidden1': 512, 'ker_reg': 1e-06, 'lr': 0.001}
Train on 460 samples, validate on 115 samples                                      
Epoch 1/1000                                                                       
 - 35s - loss: 2.6005 - acc: 0.5522 - val_loss: 0.8784 - val_acc: 0.7130           

Epoch 2/1000                                                                       
 - 0s - loss: 0.5814 - acc: 0.7978 - val_loss: 0.7454 - val_acc: 0.7478            

Epoch 3/1000                 

Epoch 17/1000                                                                      
 - 0s - loss: 2.9498 - acc: 0.6674 - val_loss: 2.6648 - val_acc: 0.7130            

Epoch 18/1000                                                                      
 - 0s - loss: 2.8985 - acc: 0.6739 - val_loss: 2.6181 - val_acc: 0.7043            

Epoch 19/1000                                                                      
 - 0s - loss: 2.8325 - acc: 0.7065 - val_loss: 2.5743 - val_acc: 0.7043            

Epoch 20/1000                                                                      
 - 0s - loss: 2.7821 - acc: 0.7304 - val_loss: 2.5305 - val_acc: 0.7217            

Epoch 21/1000                                                                      
 - 0s - loss: 2.7399 - acc: 0.7326 - val_loss: 2.4923 - val_acc: 0.7391            

Epoch 22/1000                                                                      
 - 0s - loss: 2.6739 - acc: 0.7565 - val_loss: 2.4597 - val_acc: 0.7217

Epoch 14/1000                                                                      
 - 0s - loss: 41.4065 - acc: 0.8522 - val_loss: 40.1458 - val_acc: 0.8435          

Epoch 00014: early stopping                                                        
 64/115 [===============>..............]                                           
 - ETA: 0s                                                                         
                                                                                  
115/115 [==============================]                                           
 - 0s 284us/step                                                                   

Validation Accuracy: 84.3478%                                                      
Validation Loss: 40.14575908495032                                                 
Test Accuracy: 0.8588235294117647                                                  
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

 - 0s - loss: 0.6659 - acc: 0.8413 - val_loss: 0.6448 - val_acc: 0.8261            

Epoch 12/1000                                                                      
 - 0s - loss: 0.6696 - acc: 0.8304 - val_loss: 0.6620 - val_acc: 0.8435            

Epoch 13/1000                                                                      
 - 0s - loss: 0.6462 - acc: 0.8391 - val_loss: 0.6273 - val_acc: 0.8261            

Epoch 00013: early stopping                                                        
 64/115 [===============>..............]                                           
 - ETA: 0s                                                                         
                                                                                  
115/115 [==============================]                                           
 - 0s 277us/step                                                                   

Validation Accuracy: 82.6087%                                            